In [ ]:
import os
import cv2
import math
import random
import numpy as np
import pandas as pd
import gc
from tqdm import tqdm

import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import EfficientNetB4

import matplotlib.pyplot as plt
#from sklearn.model_selection import train_test_split
#from sklearn.neighbors import NearestNeighbors
# from sklearn.feature_extraction.text import TfidfVectorizer
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
class CFG:
    seed = 2021
    Epochs = 60
    classes = 11014 
    scale = 30 
    margin = 0.1
    fc_dim = 512
    img_size = 384  
    batch_size = 16
    channels = 3

In [ ]:
# RESTRICT TENSORFLOW TO 2GB OF GPU RAM
# SO THAT WE HAVE 14GB RAM FOR RAPIDS
LIMIT = 2.0
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*LIMIT)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        #print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
print('We will restrict TensorFlow to max %iGB GPU RAM'%LIMIT)
print('then RAPIDS can use %iGB GPU RAM'%(16-LIMIT))

In [ ]:
def read_dataset(csv_path, image_path):
    df = pd.read_csv(csv_path)
    df_cu = cudf.DataFrame(df)
    image_paths = image_path + df['image']
    return df, df_cu, image_paths

In [ ]:
image_path = '../input/shopee-product-matching/test_images/'
csv_path = '../input/shopee-product-matching/test.csv' # repalce the train.csv
df, df_cu, image_paths = read_dataset(csv_path, image_path)

In [ ]:
# label2id = dict(zip(range(df.label_group.nunique()),df.label_group.unique()))
# id2label = dict(zip(df.label_group.unique(),range(df.label_group.nunique())))
# df["labels"] = df["label_group"].map(id2label)

In [ ]:
def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1

In [ ]:
def get_image_neighbors(df, embeddings, KNN=50):

    model = NearestNeighbors(n_neighbors = KNN)
    model.fit(embeddings)
    distances, indices = model.kneighbors(embeddings)
    
    threshold = 4.5
    predictions = []
    for k in tqdm(range(embeddings.shape[0])):
        idx = np.where(distances[k,] < threshold)[0]
        ids = indices[k,idx]
        posting_ids = df['posting_id'].iloc[ids].values
        predictions.append(posting_ids)
        
    del model, distances, indices
    gc.collect()
    return df, predictions

In [ ]:
# Arcmarginproduct class keras layer
class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

In [ ]:
# Function to decode our images
def preprocess_image(image):
    
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)  
    image = tf.image.resize(image, [CFG.img_size, CFG.img_size])
    return image

# Function to read our test image and return image
def load_and_preprocess_image(image, label):
    image = tf.io.read_file(image)
    image = preprocess_image(image)
    return image, label

def get_dataset(image, label):
    dataset = tf.data.Dataset.from_tensor_slices((image, label))
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls = AUTO) 
    return dataset


In [ ]:

# 1. datasets
dataset = get_dataset(image_paths, tf.zeros(shape=len(image_paths), dtype=tf.float32))
# dataset = get_dataset(image_paths, df.labels)

# split the datasets
test_ds = dataset.batch(CFG.batch_size).prefetch(AUTO)

In [ ]:
def create_model():

  margin = ArcMarginProduct(n_classes = CFG.classes, s = 30, m = 0.5, 
                name = 'head/arc_margin', dtype = 'float32')

  image = tf.keras.layers.Input(shape=(CFG.img_size, CFG.img_size, 3), name='input/image')
  label = tf.keras.layers.Input((), name='input/label')

  x = EfficientNetB4(include_top = False, weights = None)(image)
  x = tf.keras.layers.GlobalAveragePooling2D(name='head/pooling')(x)
  
  x = tf.keras.layers.BatchNormalization(name='head/bn1')(x)
  x = tf.keras.layers.Dropout(rate = 0.5, name='head/dropout')(x)
  x = tf.keras.layers.Dense(CFG.fc_dim,name='head/dense')(x)
  x = tf.keras.layers.BatchNormalization(name='head/bn2')(x)

  x = margin([x, label]) # Archead
  output = tf.keras.layers.Softmax()(x)

  model = tf.keras.models.Model(inputs = [image, label], outputs = output)

  opt = tf.keras.optimizers.Adam(learning_rate = 0.001)

  model.compile(
        optimizer = opt,
        loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
        metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
    )

  return model

In [ ]:
saved_model_path = '../input/github-model/model/effnetb4_arc.h5'

In [ ]:
reconstructed_model = create_model()
reconstructed_model.load_weights(saved_model_path)
# 重新取输入和输出 选取embedding层作为输出
reconstructed_model = tf.keras.models.Model(inputs = reconstructed_model.input[0], outputs = reconstructed_model.layers[-4].output) 
reconstructed_model.build((None, CFG.img_size, CFG.img_size, 3))
reconstructed_model.summary()

In [ ]:
embeds = []
for images, labels in tqdm(test_ds):
    embeds.append(reconstructed_model.predict(images))

In [ ]:
image_embeddings = np.concatenate(embeds)
del reconstructed_model, embeds
gc.collect()

In [ ]:
df, image_predictions = get_image_neighbors(df, image_embeddings, KNN = 50 if len(df_cu) > 3 else 3 ) # 

In [ ]:
#https://www.kaggle.com/cdeotte/part-2-rapids-tfidfvectorizer-cv-0-700#Use-Text-Embeddings
def get_text_predictions(df, df_cu, max_features=25_000):
    model = TfidfVectorizer(stop_words=None, binary=True, max_features=max_features)
    text_embeddings = model.fit_transform(df_cu['title']).toarray()
    preds = []
    CHUNK = 1024*4

    print('Finding similar titles...')
    CTS = len(df)//CHUNK
    if len(df)%CHUNK!=0: CTS += 1
    for j in range( CTS ):

        a = j*CHUNK
        b = (j+1)*CHUNK
        b = min(b,len(df))
        print('chunk',a,'to',b)

        # COSINE SIMILARITY DISTANCE
        cts = cupy.matmul(text_embeddings, text_embeddings[a:b].T).T

        for k in range(b-a):
            IDX = cupy.where(cts[k,]>0.80)[0]
            o = df.iloc[cupy.asnumpy(IDX)].posting_id.values
            
            for ii in np.arange(0.80, 0.50, -0.02):
                if ii > 0.5 and o.shape[0] <= 1:
                    IDX = cupy.where(cts[k,] > ii)[0]
                    o = df.iloc[cupy.asnumpy(IDX)].posting_id.values
            preds.append(o)
    
    del model,text_embeddings
    gc.collect()
    return preds

In [ ]:
text_predictions = get_text_predictions(df, df_cu, max_features=25000)

In [ ]:
def combine_predictions(row):
    x = np.concatenate([row['image_predictions'], row['text_predictions']])
    # x = row['image_predictions']
    return ' '.join( np.unique(x) )

In [ ]:
df['image_predictions'] = image_predictions
df['text_predictions'] = text_predictions
df['matches'] = df.apply(combine_predictions, axis=1)
df[['posting_id', 'matches']].to_csv('submission.csv', index=False)

In [ ]:
df['f1'] = f1_score(df['posting_id'], df['matches'])
df['f1'].mean()

In [ ]:
#df[['posting_id','image_predictions', 'text_predictions', 'matches', 'f1']].head()